In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Base URL
base_url = "https://www.noon.com/egypt-en/electronics-and-mobiles/mobiles-and-accessories/mobiles-20905/eg-all-mobiles/?fisCarousel=true&fis_fbn=1&limit=50&page={}&sortby=popularity&sortdir=desc"

# Initialize lists to store scraped data
product_links = []

# Pagination loop
page = 1
while True:
    # Request the page
    url = base_url.format(page)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all product containers
    products = soup.find_all('div', class_='sc-19767e73-0 bwele')

    if not products:
        break  # Stop if no products found on the page

    for product in products:

        # Extract product link
        link = product.find('a', href=True)['href']
        full_link = f"https://www.noon.com{link}"  # Make sure the URL is complete
        product_links.append(full_link)

    # Increment page count
    page += 1

# Create DataFrame
df = pd.DataFrame({

    'Product Link': product_links
})

# Export to CSV
df.to_csv('noon_mobile_links.csv', index=False)



Scraping completed and data saved to 'noon_mobile_links.csv'


In [ ]:
# Load the CSV file with URLs
file_path = '/content/noon_mobile_links.csv'
urls_df = pd.read_csv(file_path)

# Prepare an empty list to collect data
all_data = []

# Iterate over each URL in the DataFrame
for url in urls_df['Product Link']:
    # Send a GET request to the webpage
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extract the title
    title = soup.find('h1', class_='sc-de5a1c50-17 dJnofv').text.strip()

    # Extract the price
    price = soup.find('div', class_='priceNow').text.strip() if soup.find('div', class_='priceNow') else 'N/A'

    # Extract the availability
    availability = 'Available' if soup.find('button', class_='sc-qGWeR') else 'Out of stock'

    # Extract the specifications table
    specifications = {}
    specs_table = soup.find('table').find('tbody').find_all('tr')

    for row in specs_table:
        # Get the header (first column) and value (second column)
        header = row.find_all('td')[0].text.strip()
        value = row.find_all('td')[1].text.strip()
        specifications[header] = value

    # Add title, price, and availability to specifications
    specifications['Title'] = title
    specifications['Price'] = price
    specifications['Availability'] = availability

    # Append the specifications as a row to the list
    all_data.append(specifications)

# Convert the list of dictionaries to a DataFrame
specs_df = pd.DataFrame(all_data)

# Save the data to a CSV file
specs_df.to_csv('/content/noon_mobile_Data.csv', index=False)



'/content/noon_mobile_Data.csv'